### Neuron identification and density calculation

*Step (d) of the block diagram of the methods section.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import natsort
import scipy.ndimage as ndi
import skimage
from skimage.feature import blob_log
import cv2
import json
from PIL import Image

def open_images(img_folder, ROI_folder, file, plot=False):
    
    filename = file.stem
    img = Image.open(img_folder/file)
    img = np.array(img)
    img_ROI = Image.open(ROI_folder/f'{filename}.png')
    img_ROI = np.array(img_ROI)
        
    if plot:
        plt.figure(figsize=[10,8])
        plt.subplot(1, 2, 1)
        plt.imshow(img, 'gray')
        plt.subplot(1, 2, 2)
        plt.imshow(img_ROI, 'gray')
            
    return img, img_ROI

def get_quantification_ROI(img_ROI):
    '''Get distance transform from cortex surface.'''

    img_ROI_bin = img_ROI>0

    # Find cortex surface from ROI.
    contours, _ = cv2.findContours(img_ROI_bin.astype(np.uint8), mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
    contour = contours[0][:,0]
    contour = contour[:,::-1]
    inside = False
    surface_cont = []
    for point in contour:
        if img_ROI[tuple(point)]==1:
            inside = not inside
        if inside:
            surface_cont.append(point)
    surface_cont = np.array(surface_cont)
    
    img_surface = np.zeros_like(img_ROI, dtype=np.uint8)
    for point in surface_cont:
        img_surface[tuple(point)] = 1
    img_surface_inv = np.logical_not(img_surface)
    img_dist = ndi.distance_transform_edt(img_surface_inv)
    
    return img_dist

def detect_nuclei(img, img_dist, sub_factor=1, min_sigma=3, max_sigma=6, num_sigma=4, log_threshold=0.05, overlap=0.25,
                  sig_x0=100, sig_k=0.05, apply_sigmoid=True, return_img=True):
    '''Detect neuron nuclei in image using a Laplacian of Gaussian filter.'''

    img_diff = img.copy()
    img_diff -= np.min(img_diff)
    img_diff = img_diff/float(np.max(img_diff))

    if apply_sigmoid:
        # Not used
        img_sig = 1./(1+np.exp(-sig_k*(img_dist-sig_x0)))
        img_nuclei = img_sig*img_diff
    else:
        img_nuclei = img_diff

    blobs = blob_log(img_nuclei, min_sigma=min_sigma, max_sigma=max_sigma, num_sigma=num_sigma, 
                     threshold=log_threshold, overlap=overlap)
    
    blobs_pos = blobs[:, :2].astype(int)
    blobs_radius = blobs[:, 2]*np.sqrt(2)

    if return_img:
        img_nuclei = np.zeros_like(img_nuclei, dtype=np.uint8)
        img_nuclei[blobs_pos[:,0], blobs_pos[:,1]] = 1
        return img_nuclei
    else:
        return blobs_pos, blobs_radius

def get_shell(img, img_dist, shell_width, index):
    '''Get ROI of distance interval from cortex surface.'''
    
    first_dist = shell_width*index
    last_dist = shell_width*(index+1)
    img_shell = (img_dist>first_dist) & (img_dist<=last_dist)
    img_in_shell = img*img_shell
    
    return img_in_shell, img_shell
    
def write_data(nuc_density_all, filename, folder=None):
    
    if folder is None:
        folder = './'
    elif folder[-1]!='/':
        folder += '/'
    
    shell_width = nuc_density_all['info']['shell_width_um']
    max_num_shells = 0
    for k, v in nuc_density_all.items():
        if k != 'info':
            s = len(v)
            if s>max_num_shells:
                max_num_shells = s

    fd = open(f'{folder}{filename}.csv', 'w')
    line = 'Sample name;'
    for i in range(max_num_shells-1):
        line += f'{i*shell_width}-{(i+1)*shell_width};'
    line += f'{(max_num_shells-1)*shell_width}-{(max_num_shells)*shell_width}\n'
    fd.write(line)
    for k, v in nuc_density_all.items():
        if k != 'info':
            line = f'{k};'
            c_data = v
            for value in c_data[:-1]:
                line += f'{value:.1f};'
            line += f'{c_data[-1]:.1f}\n'
            fd.write(line)
    fd.close()
    
def verify_images(nuclei_centers_dil, nuclei_centers):
    
    img_label_dil, num_comp_dil = ndi.label(nuclei_centers_dil)
    img_label, num_comp = ndi.label(nuclei_centers)
    if num_comp_dil!=num_comp:
        return False
    
    tam_comps = ndi.sum(nuclei_centers, img_label, range(1, num_comp+1))
    if np.sum(tam_comps>1)>0:
        return False
    
    return True

In [2]:
data_folder = Path('neuron')
img_folder = data_folder/'images'
ROI_folder = data_folder/'masks - Copy'
out_folder = Path('neuron/density')

files = os.listdir(img_folder)
files = natsort.natsorted(files)

scale = 0.454       # microns/px
shell_width_um = 50 # distance intervals to calculate density, in microns

### Neuron detection

In [ ]:
for file in files:
    
    file_key = file.split('.')[0]
    
    img, img_ROI = open_images(img_folder, ROI_folder, img_folder/file)
    img_ROI_bin = img_ROI>0

    img_dist = get_quantification_ROI(img_ROI)
    img_dist_ROI = img_dist*img_ROI_bin
    
    img_eq = skimage.exposure.equalize_adapthist(img, kernel_size=151, clip_limit=0.03)

    # Nuclei detection
    nuclei_centers = detect_nuclei(img_eq, img_dist, min_sigma=10, max_sigma=20, num_sigma=8, log_threshold=0.1, overlap=0.6, apply_sigmoid=False)
    nuclei_centers = nuclei_centers*img_ROI_bin
    
    nuclei_centers_dil = ndi.binary_dilation(nuclei_centers, iterations=2)
    nuclei_centers_save = np.zeros(nuclei_centers_dil.shape+(3,), dtype=np.uint8)
    nuclei_centers_save[:,:,0] = 255*nuclei_centers_dil
    
    Image.fromarray(nuclei_centers_save).save(out_folder/f'detected_nuclei/{file_key}.png')

### Neuron density calculation

In [ ]:
shell_width = shell_width_um/scale

nuc_density_all = {'info':{'scale':scale, 'shell_width_um':shell_width_um}}
areas_all = {}
for file in files:
    
    print(file)
    file_key = file.split('.')[0]
    
    img, img_ROI = open_images(img_folder, ROI_folder, img_folder/file)
    img_ROI_bin = img_ROI>0

    # Distances from cortex surface
    img_dist = get_quantification_ROI(img_ROI)
    img_dist_ROI = img_dist*img_ROI_bin
    
    img_eq = skimage.exposure.equalize_adapthist(img, kernel_size=151, clip_limit=0.03)

    # Detected nuclei
    nuclei_centers_dil = np.array(Image.open(out_folder/f'detected_nuclei/{file_key}.png'))[:,:,0]
    nuclei_centers = ndi.binary_erosion(nuclei_centers_dil, iterations=2)    
    
    if not verify_images(nuclei_centers_dil, nuclei_centers):
        print(f'Problem with image {file}')
    
    num_full_shells = int(np.floor(np.max(img_dist_ROI)/shell_width))
    if np.max(img_dist_ROI)/shell_width-num_full_shells>0:
        num_shells = num_full_shells + 1
    else:
        num_shells = num_full_shells

    # For each distance interval (shell) from cortex surface, calculate nuclei density.
    nuc_density = []
    areas = []
    for i in range(num_shells):
        img_nuc_shell, img_shell = get_shell(nuclei_centers, img_dist_ROI, shell_width, i)
        area = np.sum(img_shell)*scale**2*1e-6
        if area<0.004:   # ROI area is too small for good statistics
            nuc_density.append(-1)
        else:
            nuc_density.append(np.sum(img_nuc_shell)/area)
        areas.append(area)

    nuc_density_all[file_key] = nuc_density
    areas_all[file_key] = areas

json.dump(nuc_density_all, open('nuclei_density.json', 'w'), indent=2)
write_data(nuc_density_all, 'results', './')